In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import to_rgba
from mpl_toolkits.mplot3d import Axes3D
import matplotlib

matplotlib.rc('font', size=14)
matplotlib.rc('axes', titlesize=14)


In [ ]:
%matplotlib notebook

path = "results/"
#metrics = ["Accuracy", "Macro F1", "Micro F1", "Log loss", "Brier score"]
metrics = ["Accuracy", "F1", "F1", "Log loss", "Brier score"]

# Fig A
## Setting
To control:
- generative model: simple or complex
- sampling or deterministic
  - No. of dataset already computed
- Real model result's as maximum or not

In [ ]:
css = np.array([2,3,4,5])
wsp = np.array([0.5,1,2,5])
n_models = 30
n_datasets_gen = 30
n_datasets = 30

#kdb=1; gm="sg" #simple gen model
kdb=4; gm="cg" #complex gen model
n_rep = 1; directory = "figA_determ/"
#n_rep = 5; directory = "figA_sampling/"
#prev_name="Diff. "; real_tsdata = -1 # any from 0 to 4 ; -1 means no max.
prev_name="Rel. diff. "; real_tsdata = 0 

## Collect results with only fully labeled data

In [ ]:
mResultsFull = np.loadtxt(path+'res_exp_figA_'+gm+'_fully_labeled.resout', delimiter=",")

## Collect results with real model

In [ ]:
mResultsReal = np.loadtxt(path+'res_exp_figA_'+gm+'_real_model.resout', delimiter=",")

## Collect data for `honest` scenario

In [ ]:
Z1_complete = []
s = 0

for m in np.arange(n_models):
    for d in np.arange(n_datasets):
        id_mrf = m*n_datasets_gen+d
        id_mrr = (m*n_datasets_gen+d)*5+real_tsdata
        for c in css:
            orig_data = np.loadtxt(path+directory+'res_exp_figA_m_' + str(m) + 
                                                                '_d_' + str(d) + 
                                                                '_css_' + str(c) + 
                                                                '_k_' + str(kdb) + 
                                                                '_s_' + str(s) + 
                                                                 '.csv', delimiter=",")
            for v in np.arange(len(wsp)):
                vals = orig_data[np.arange(v, orig_data.shape[0], len(wsp)),:]
                vals = np.mean(vals, axis=0) - mResultsFull[id_mrf] # mean over repetitions and remove real values
                if real_tsdata >= 0: vals /= (mResultsReal[id_mrr] - mResultsFull[id_mrf])
                Z1_complete.append( vals ) 
Z1_complete = np.array(Z1_complete)

## Collect data for `misleading` scenario

In [ ]:
Z2_complete = []
s = 1

for m in np.arange(n_models):
    for d in np.arange(n_datasets):
        id_mrf = m*n_datasets_gen+d
        id_mrr = (m*n_datasets_gen+d)*5+real_tsdata
        for c in css:
            orig_data = np.loadtxt(path+directory+'res_exp_figA_m_' + str(m) + 
                                                                '_d_' + str(d) + 
                                                                '_css_' + str(c) + 
                                                                '_k_' + str(kdb) + 
                                                                '_s_' + str(s) + 
                                                                 '.csv', delimiter=",")
            for v in np.arange(len(wsp)):
                vals = orig_data[np.arange(v, orig_data.shape[0], len(wsp)),:]
                vals = np.mean(vals, axis=0) - mResultsFull[id_mrf] # mean over repetitions and remove real values
                if real_tsdata >= 0: vals /= (mResultsReal[id_mrr] - mResultsFull[id_mrf])
                Z2_complete.append( vals ) 
Z2_complete = np.array(Z2_complete)

## Plot

In [ ]:
%matplotlib widget
## 0:Acc, 1:Macro F1, 2:Micro F1, 3:log loss, 4:Brier score
metric_ind = 1
transparencia = 0.6


Z1 = Z1_complete[:,metric_ind]
Z1.shape = (n_models*n_datasets,len(css)*len(wsp))
Z1 = Z1.mean(axis=0)
Z1.shape = (len(css),len(wsp))
Z1 = np.transpose(Z1)
Z1p = (Z1 - np.min(Z1))/(np.max(Z1) - np.min(Z1))

Z2 = Z2_complete[:,metric_ind]
Z2.shape = (n_models*n_datasets,len(css)*len(wsp))
Z2 = Z2.mean(axis=0)
Z2.shape = (len(css),len(wsp))
Z2 = np.transpose(Z2)
Z2p = (Z2 - np.min(Z2))/(np.max(Z2) - np.min(Z2))


x = css
y_labels = wsp
y = np.arange(len(y_labels))
X, Y = np.meshgrid(x, y)


C1 = np.empty_like(Z1, dtype=object)
C2 = np.empty_like(Z2, dtype=object)
cmap1 = plt.get_cmap("spring")
cmap2 = plt.get_cmap("winter")

for i in range(Z1.shape[0]):
    for j in range(Z1.shape[1]):
        C1[i,j] = to_rgba(cmap1(Z1p[i,j]), transparencia)
        C2[i,j] = to_rgba(cmap2(Z2p[i,j]), transparencia)


# Create a transparent bridge region
X_bridge = np.vstack([X[-1,:],X[-1,:]])
Y_bridge = np.vstack([Y[-1,:],Y[-1,:]])
Z_bridge = np.vstack([Z1[-1,:],Z2[-1,:]])
color_bridge = np.empty_like(Z_bridge, dtype=object)

color_bridge.fill((1,1,1,0)) # RGBA colour, onlt the last component matters.

# Join the two surfaces flipping one of them (using also the bridge)
X_full = np.vstack([X, X_bridge, np.flipud(X)])
Y_full = np.vstack([Y, Y_bridge, np.flipud(Y)])
Z_full = np.vstack([Z1, Z_bridge, np.flipud(Z2)])
color_full = np.vstack([C1, color_bridge, np.flipud(C2)])


### print figure
fig = plt.figure(figsize=(10,5))
ax = fig.gca(projection='3d')

surf_full = ax.plot_surface(X_full, Y_full, Z_full, rstride=1, cstride=1,
                            facecolors=color_full, linewidth=0,
                            antialiased=False, shade=False)

ax.set_xlabel("Size of the candidate sets")
ax.set_ylabel("Prop. weakly labeled ex.")
ax.set_zlabel(prev_name+metrics[metric_ind])

ax.set_xticks(x)
ax.set_xticklabels(x)
ax.set_yticks(y)
ax.set_yticklabels(y_labels)

ax.text(3, 3, Z1[0,-1], "Honest", zdir="x",
        color=cmap1(Z1p[0,-1]))

ax.text(0, 3, Z2[-1,-1], "Misleading", zdir="x",
        color=cmap2(Z2p[0,-1]))


# manipulate view
ax.view_init(20, -60)

plt.show()

# FigB

## Setting
To control:
- generative model: simple or complex
- sampling or deterministic
  - No. of dataset already computed
- Real model result's as maximum or not

In [ ]:
ind_fss = np.arange(10)#np.array([0,1,3,6,9])
fss = np.round((1+np.arange(10))*(100.0/3)).astype(int)
fss = fss[ind_fss]
n_fss = 10
wsp = np.array([0.5,1,2,5])
n_models = 30
n_datasets_gen = 30
n_datasets = 30

#kdb=1; gm="sg" #simple gen model
kdb=4; gm="cg" #complex gen model
n_rep = 1; directory = "figB_determ/"
#n_rep = 5; directory = "figB_sampling/"
#prev_name="Diff. "; real_tsdata = -1 # any from 0 to 4 ; -1 means no max.
prev_name="Rel. diff. "; real_tsdata = 0 

## Collect results with only fully labeled data

In [ ]:
mResultsFull = np.loadtxt(path+'res_exp_figB_'+gm+'_fully_labeled.resout', delimiter=",")

## Collect results with real model

In [ ]:
mResultsReal = np.loadtxt(path+'res_exp_figB_'+gm+'_real_model.resout', delimiter=",")

## Collect data for `honest` scenario

In [ ]:
Z1_complete = []
s = 0

for m in np.arange(n_models):
    for d in np.arange(n_datasets):
        for f in ind_fss:
            orig_data = np.loadtxt(path+directory+'res_exp_figB_m_' + str(m) + 
                                                                '_d_' + str(d) + 
                                                                '_fss_' + str(f) + 
                                                                '_k_' + str(kdb) + 
                                                                '_s_' + str(s) + 
                                                                 '.csv', delimiter=",")
            id_mrf = (m*n_datasets_gen+d)*n_fss+f
            id_mrr = ((m*n_datasets_gen+d)*n_fss+f)*5+real_tsdata
            for v in np.arange(len(wsp)):
                vals = orig_data[np.arange(v, orig_data.shape[0], len(wsp)),:]
                vals = np.mean(vals, axis=0) - mResultsFull[id_mrf] # mean over repetitions and remove real values
                if real_tsdata >= 0: vals /= (mResultsReal[id_mrr] - mResultsFull[id_mrf])
                Z1_complete.append( vals ) 
Z1_complete = np.array(Z1_complete)

## Collect data for `misleading` scenario

In [ ]:
Z2_complete = []
s = 1

for m in np.arange(n_models):
    for d in np.arange(n_datasets):
        for f in ind_fss:
            orig_data = np.loadtxt(path+directory+'res_exp_figB_m_' + str(m) + 
                                                                '_d_' + str(d) + 
                                                                '_fss_' + str(f) + 
                                                                '_k_' + str(kdb) + 
                                                                '_s_' + str(s) + 
                                                                 '.csv', delimiter=",")
            id_mrf = (m*n_datasets_gen+d)*n_fss+f
            id_mrr = ((m*n_datasets_gen+d)*n_fss+f)*5+real_tsdata
            for v in np.arange(len(wsp)):
                vals = orig_data[np.arange(v, orig_data.shape[0], len(wsp)),:]
                vals = np.mean(vals, axis=0) - mResultsFull[id_mrf] # mean over repetitions and remove real values
                if real_tsdata >= 0: vals /= (mResultsReal[id_mrr] - mResultsFull[id_mrf])
                Z2_complete.append( vals ) 
            #mResults.append( final_data )
Z2_complete = np.array(Z2_complete)

## Plot

In [ ]:
%matplotlib widget
## 0:Acc, 1:Macro F1, 2:Micro F1, 3:log loss, 4:Brier score
metric_ind = 4
transparencia = 0.6


Z1 = Z1_complete[:,metric_ind]
Z1.shape = (n_models*n_datasets,len(fss)*len(wsp))
Z1 = Z1.mean(axis=0)
Z1.shape = (len(fss),len(wsp))
Z1 = np.transpose(Z1)
Z1p = (Z1 - np.min(Z1))/(np.max(Z1) - np.min(Z1))

Z2 = Z2_complete[:,metric_ind]
Z2.shape = (n_models*n_datasets,len(fss)*len(wsp))
Z2 = Z2.mean(axis=0)
Z2.shape = (len(fss),len(wsp))
Z2 = np.transpose(Z2)
Z2p = (Z2 - np.min(Z2))/(np.max(Z2) - np.min(Z2))


x = np.arange(len(fss))
x_labels = fss
y = np.arange(len(wsp))
y_labels = wsp

X, Y = np.meshgrid(x, y)


C1 = np.empty_like(Z1, dtype=object)
C2 = np.empty_like(Z2, dtype=object)
cmap1 = plt.get_cmap("spring")
cmap2 = plt.get_cmap("winter")

for i in range(Z1.shape[0]):
    for j in range(Z1.shape[1]):
        C1[i,j] = to_rgba(cmap1(Z1p[i,j]), transparencia)
        C2[i,j] = to_rgba(cmap2(Z2p[i,j]), transparencia)


# Create a transparent bridge region
X_bridge = np.vstack([X[-1,:],X[-1,:]])
Y_bridge = np.vstack([Y[-1,:],Y[-1,:]])
Z_bridge = np.vstack([Z1[-1,:],Z2[-1,:]])
color_bridge = np.empty_like(Z_bridge, dtype=object)

color_bridge.fill((1,1,1,0)) # RGBA colour, onlt the last component matters.

# Join the two surfaces flipping one of them (using also the bridge)
X_full = np.vstack([X, X_bridge, np.flipud(X)])
Y_full = np.vstack([Y, Y_bridge, np.flipud(Y)])
Z_full = np.vstack([Z1, Z_bridge, np.flipud(Z2)])
color_full = np.vstack([C1, color_bridge, np.flipud(C2)])


### print figure
fig = plt.figure(figsize=(10,5))
ax = fig.gca(projection='3d')

surf_full = ax.plot_surface(X_full, Y_full, Z_full, rstride=1, cstride=1,
                            facecolors=color_full, linewidth=0,
                            antialiased=False, shade=False)

ax.set_xlabel("Size of the fully labeled set")
ax.set_ylabel("Prop. weakly labeled ex.")
ax.set_zlabel(prev_name+metrics[metric_ind])

#fig.colorbar(surf_full, shrink=0.5, aspect=25, label="DAV")
ax.set_xticks(x)
plt.xticks(rotation = 45)
ax.set_xticklabels(x_labels)
ax.set_yticks(y)
ax.set_yticklabels(y_labels)

ax.text(3, 3, Z1[3,3], "Honest", zdir="x",
        color=cmap1(Z1p[0,-1]))

ax.text(3, 3, Z2[3,3], "Misleading", zdir="x",
        color=cmap2(Z2p[0,-1]))

# manipulate view
ax.view_init(10, -80)

plt.show()

# FigC-1

## Setting
To control:
- generative model: simple or complex
- sampling or deterministic
  - No. of dataset already computed (different from FigA and FigC)
- Real model result's as maximum or not

In [ ]:
css = np.array([2,3,4,5])
amb = np.array([0.25,0.5,0.75,1.0])
wsp_ind = 2; wsp = np.array([0.5,1,2,5])
n_models = 30
n_datasets_gen = 30
n_datasets_Z1 = 30 
n_datasets_Z2 = 30

#kdb=1; gm="sg" #simple gen model
kdb=4; gm="cg" #complex gen model
n_rep = 1; directory_Z1 = "figC_determ/"; directory_Z2 = "figA_determ/"
#n_rep = 5; directory_Z1 = "figC_sampling/"; directory_Z2 = "figA_sampling/"
#prev_name="Diff. "; real_tsdata = -1 # any from 0 to 4 ; -1 means no max.
prev_name="Rel. diff. "; real_tsdata = 0 

## Collect results with only fully labeled data

In [ ]:
mResultsFull = np.loadtxt(path+'res_exp_figA_'+gm+'_fully_labeled.resout', delimiter=",")

## Collect results with real model

In [ ]:
mResultsReal = np.loadtxt(path+'res_exp_figA_'+gm+'_real_model.resout', delimiter=",")

## Collect data for `consistent-labeling` scenario

In [ ]:
Z1_complete = []

for m in np.arange(n_models):
    for d in np.arange(n_datasets_Z1):
        id_mrf = m*n_datasets_gen+d
        id_mrr = (m*n_datasets_gen+d)*5+real_tsdata
        for c in css:
            for a in np.arange(len(amb))+1:
                orig_data = np.loadtxt(path+directory_Z1+'res_exp_figC_m_' + str(m) + 
                                                                    '_d_' + str(d) + 
                                                                    '_css_' + str(c) + 
                                                                    '_k_' + str(kdb) + 
                                                                    '_s_' + str(a) + 
                                                                     '.csv', delimiter=",")
                vals = orig_data[np.arange(wsp_ind, orig_data.shape[0], len(wsp)),:]
                vals = np.mean(vals, axis=0) - mResultsFull[id_mrf] # mean over repetitions and remove real values
                if real_tsdata >= 0: vals /= (mResultsReal[id_mrr] - mResultsFull[id_mrf])
                Z1_complete.append( vals ) 
Z1_complete = np.array(Z1_complete)

## Collect data for `inconsistent-labeling` scenario (baseline)


In [ ]:
Z2_complete = []
s = 0 # honest scenario

for m in np.arange(n_models):
    for d in np.arange(n_datasets_Z2):
        id_mrf = m*n_datasets_gen+d
        id_mrr = (m*n_datasets_gen+d)*5+real_tsdata
        for c in css:
            orig_data = np.loadtxt(path+directory_Z2+'res_exp_figC_m_' + str(m) + 
                                                                '_d_' + str(d) + 
                                                                '_css_' + str(c) + 
                                                                '_k_' + str(kdb) + 
                                                                '_s_' + str(s) + 
                                                                 '.csv', delimiter=",")
            vals = orig_data[np.arange(wsp_ind, orig_data.shape[0], len(wsp)),:]
            vals = np.mean(vals, axis=0) - mResultsFull[id_mrf] # mean over repetitions and remove real values
            if real_tsdata >= 0: vals /= (mResultsReal[id_mrr] - mResultsFull[id_mrf])
            Z2_complete.append( vals ) 
Z2_complete = np.array(Z2_complete)

## Plot

In [ ]:
%matplotlib widget
## 0:Acc, 1:Macro F1, 2:Micro F1, 3:log loss, 4:Brier score
metric_ind = 1
transparencia = 0.6


Z1 = Z1_complete[:,metric_ind]
Z1.shape = (n_models*n_datasets_Z1,len(css)*len(amb))
Z1 = Z1.mean(axis=0)
Z1.shape = (len(css),len(amb))
Z1 = np.transpose(Z1)
Z1p = (Z1 - np.min(Z1))/(np.max(Z1) - np.min(Z1))

Z2 = Z2_complete[:,metric_ind]
Z2.shape = (n_models*n_datasets_Z2,len(css))
Z2 = Z2.mean(axis=0)
Z2 = np.repeat(Z2, len(amb)) # dimensions of Z1 and Z2 are different!!! dim(Z1) = dim(Z2)*4
Z2.shape = (len(css),len(amb))
Z2 = np.transpose(Z2)
Z2p = (Z2 - np.min(Z2))/(np.max(Z2) - np.min(Z2))


x = np.arange(len(css))
x_labels = css
y = np.arange(len(amb))
y_labels = amb

X, Y = np.meshgrid(x, y)


C1 = np.empty_like(Z1, dtype=object)
C2 = np.empty_like(Z2, dtype=object)
cmap1 = plt.get_cmap("spring")
cmap2 = plt.get_cmap("winter")

for i in range(Z1.shape[0]):
    for j in range(Z1.shape[1]):
        C1[i,j] = to_rgba(cmap1(Z1p[i,j]), transparencia)
        C2[i,j] = to_rgba(cmap2(Z2p[i,j]), transparencia)


# Create a transparent bridge region
X_bridge = np.vstack([X[-1,:],X[-1,:]])
Y_bridge = np.vstack([Y[-1,:],Y[-1,:]])
Z_bridge = np.vstack([Z1[-1,:],Z2[-1,:]])
color_bridge = np.empty_like(Z_bridge, dtype=object)

color_bridge.fill((1,1,1,0)) # RGBA colour, onlt the last component matters.

# Join the two surfaces flipping one of them (using also the bridge)
X_full = np.vstack([X, X_bridge, np.flipud(X)])
Y_full = np.vstack([Y, Y_bridge, np.flipud(Y)])
Z_full = np.vstack([Z1, Z_bridge, np.flipud(Z2)])
color_full = np.vstack([C1, color_bridge, np.flipud(C2)])


### print figure
fig = plt.figure(figsize=(10,5))
ax = fig.gca(projection='3d')

surf_full = ax.plot_surface(X_full, Y_full, Z_full, rstride=1, cstride=1,
                            facecolors=color_full, linewidth=0,
                            antialiased=False, shade=False)

ax.set_xlabel("Size of candidate sets")
ax.set_ylabel("Prob. of co-occurrence")
ax.set_zlabel(prev_name+metrics[metric_ind])

ax.set_xticks(x)
ax.set_xticklabels(x_labels)
ax.set_yticks(y)
ax.set_yticklabels(y_labels)

ax.text(3, 3, Z1[-1,0], "With co-occurrence", zdir="x",
        color=cmap1(Z1p[0,-1]))

ax.text(3, 3, Z2[-1,0], "Without", zdir="x",
        color=cmap2(Z2p[0,-1]))


# manipulate view
ax.view_init(20, 30)

plt.show()

# FigC-2

## Setting
To control:
- generative model: simple or complex
- sampling or deterministic
  - No. of dataset already computed (different from FigA and FigC)
- Real model result's as maximum or not

In [ ]:
css_ind = 0; css = np.array([2,3,4,5])
amb = np.array([0.25,0.5,0.75,1.0])
wsp = np.array([0.5,1,2,5])
n_models = 30
n_datasets_gen = 30
n_datasets_Z1 = 30
n_datasets_Z2 = 30

kdb=1; gm="sg" #simple gen model
#kdb=4; gm="cg" #complex gen model
n_rep = 1; directory_Z1 = "figC_determ/"; directory_Z2 = "figA_determ/"
#n_rep = 5; directory_Z1 = "figC_sampling/"; directory_Z2 = "figA_sampling/"
#prev_name="Diff. "; real_tsdata = -1 # any from 0 to 4 ; -1 means no max.
prev_name="Rel. diff. "; real_tsdata = 0 

## Collect results with only fully labeled data

In [ ]:
mResultsFull = np.loadtxt(path+'res_exp_figA_'+gm+'_fully_labeled.resout', delimiter=",")

## Collect results with real model

In [ ]:
mResultsReal = np.loadtxt(path+'res_exp_figA_'+gm+'_real_model.resout', delimiter=",")

## Collect data for `consistent-labeling` scenario

In [ ]:
Z1_complete = []
inta = 0
for m in np.arange(n_models):
    for d in np.arange(n_datasets_Z1):
        id_mrf = m*n_datasets_gen+d
        id_mrr = (m*n_datasets_gen+d)*5+real_tsdata
        for a in np.arange(len(amb))+1:
            orig_data = np.loadtxt(path+directory_Z1+'res_exp_figC_m_' + str(m) + 
                                                                '_d_' + str(d) + 
                                                                '_css_' + str(css[css_ind]) + 
                                                                '_k_' + str(kdb) + 
                                                                '_s_' + str(a) + 
                                                                 '.csv', delimiter=",")
            for v in np.arange(len(wsp)):
                vals = orig_data[np.arange(v, orig_data.shape[0], len(wsp)),:]
                vals = np.mean(vals, axis=0) - mResultsFull[id_mrf] # mean over repetitions and remove real values
                if real_tsdata >= 0: vals /= (mResultsReal[id_mrr] - mResultsFull[id_mrf])
                Z1_complete.append( vals ) 
                inta+=1
Z1_complete = np.array(Z1_complete)

## Collect data for `inconsistent-labeling` scenario (baseline)


In [ ]:
Z2_complete = []
s = 0 # honest scenario

for m in np.arange(n_models):
    for d in np.arange(n_datasets_Z2):
        id_mrf = m*n_datasets_gen+d
        id_mrr = (m*n_datasets_gen+d)*5+real_tsdata
        orig_data = np.loadtxt(path+directory_Z2+'res_exp_figA_m_' + str(m) + 
                                                            '_d_' + str(d) + 
                                                            '_css_' + str(css[css_ind]) + 
                                                            '_k_' + str(kdb) + 
                                                            '_s_' + str(s) + 
                                                             '.csv', delimiter=",")
        for v in np.arange(len(wsp)):
            vals = orig_data[np.arange(v, orig_data.shape[0], len(wsp)),:]
            vals = np.mean(vals, axis=0) - mResultsFull[id_mrf] # mean over repetitions and remove real values
            if real_tsdata >= 0: vals /= (mResultsReal[id_mrr] - mResultsFull[id_mrf])
            Z2_complete.append( vals ) 
Z2_complete = np.array(Z2_complete)

## Plot

In [ ]:
%matplotlib widget
## 0:Acc, 1:Macro F1, 2:Micro F1, 3:log loss, 4:Brier score
metric_ind = 4
transparencia = 0.6


Z1 = Z1_complete[:,metric_ind]
Z1.shape = (n_models*n_datasets_Z1,len(amb)*len(wsp))
Z1 = Z1.mean(axis=0)
Z1.shape = (len(amb),len(wsp))
Z1 = np.transpose(Z1)
Z1p = (Z1 - np.min(Z1))/(np.max(Z1) - np.min(Z1))

Z2 = Z2_complete[:,metric_ind]
Z2.shape = (n_models*n_datasets_Z2,len(wsp))
Z2 = Z2.mean(axis=0)
Z2 = np.tile(Z2, len(amb)) # dimensions of Z1 and Z2 are different!!! dim(Z1) = dim(Z2)*4
Z2.shape = (len(amb),len(wsp))
Z2 = np.transpose(Z2)
Z2p = (Z2 - np.min(Z2))/(np.max(Z2) - np.min(Z2))


x = np.arange(len(amb))
x_labels = amb
y = np.arange(len(wsp))
y_labels = wsp

X, Y = np.meshgrid(x, y)


C1 = np.empty_like(Z1, dtype=object)
C2 = np.empty_like(Z2, dtype=object)
cmap1 = plt.get_cmap("spring")
cmap2 = plt.get_cmap("winter")

for i in range(Z1.shape[0]):
    for j in range(Z1.shape[1]):
        C1[i,j] = to_rgba(cmap1(Z1p[i,j]), transparencia)
        C2[i,j] = to_rgba(cmap2(Z2p[i,j]), transparencia)


# Create a transparent bridge region
X_bridge = np.vstack([X[-1,:],X[-1,:]])
Y_bridge = np.vstack([Y[-1,:],Y[-1,:]])
Z_bridge = np.vstack([Z1[-1,:],Z2[-1,:]])
color_bridge = np.empty_like(Z_bridge, dtype=object)

color_bridge.fill((1,1,1,0)) # RGBA colour, onlt the last component matters.

# Join the two surfaces flipping one of them (using also the bridge)
X_full = np.vstack([X, X_bridge, np.flipud(X)])
Y_full = np.vstack([Y, Y_bridge, np.flipud(Y)])
Z_full = np.vstack([Z1, Z_bridge, np.flipud(Z2)])
color_full = np.vstack([C1, color_bridge, np.flipud(C2)])


### print figure
fig = plt.figure(figsize=(10,5))
ax = fig.gca(projection='3d')

surf_full = ax.plot_surface(X_full, Y_full, Z_full, rstride=1, cstride=1,
                            facecolors=color_full, linewidth=0,
                            antialiased=False, shade=False)

ax.set_xlabel("Prob. of co-occurrence")
ax.set_ylabel("Prop. weakly labeled ex.")
ax.set_zlabel(prev_name+metrics[metric_ind])

ax.set_xticks(x)
ax.set_xticklabels(x_labels)
ax.set_yticks(y)
ax.set_yticklabels(y_labels)

ax.text(3, 3, Z1[-1,0], "With co-occurrence", zdir="x",
        color=cmap1(Z1p[0,-1]))

ax.text(3, 3, Z2[-1,0], "Without", zdir="x",
        color=cmap2(Z2p[0,-1]))


# manipulate view
ax.view_init(20, -60)

plt.show()